* https://www.kaggle.com/c/fake-news-pair-classification-challenge/data
* https://github.com/fxsjy/jieba

Embedding options
* https://github.com/Embedding/Chinese-Word-Vectors (Embedding 1)
* https://github.com/facebookresearch/fastText/blob/master/docs/crawl-vectors.md (Embedding 2)

* Performance
    * Using Embedding 1: Performance 0.7
    * Using Embedding 2: Performance 0.7

In [1]:
import glob
import pandas as pd

In [4]:
train_path = '../data/train.csv'
test_path = '../data/test.csv'
sample_submission_path = '../data/sample_submission.csv'

In [6]:
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
df_samplesub = pd.read_csv(sample_submission_path)
df_train.head(10)

,id,tid1,tid2,title1_zh,title2_zh,title1_en,title2_en,label
0,0,0,1,2017养老保险又新增两项，农村老人人人可申领，你领到了吗,警方辟谣“鸟巢大会每人领5万” 仍有老人坚持进京,There are two new old-age insurance benefits f...,"Police disprove ""bird's nest congress each per...",unrelated
1,3,2,3,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",深圳GDP首超香港？深圳统计局辟谣：只是差距在缩小,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP outstrips Hong Kong? Shenzhen S...,unrelated
2,1,2,4,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",GDP首超香港？深圳澄清：还差一点点……,"""If you do not come to Shenzhen, sooner or lat...",The GDP overtopped Hong Kong? Shenzhen clarifi...,unrelated
3,2,2,5,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",去年深圳GDP首超香港？深圳统计局辟谣：还差611亿,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP topped Hong Kong last year? She...,unrelated
4,9,6,7,"""用大蒜鉴别地沟油的方法,怎么鉴别地沟油",吃了30年食用油才知道，一片大蒜轻松鉴别地沟油,"""How to discriminate oil from gutter oil by me...",It took 30 years of cooking oil to know that o...,agreed
5,4,2,8,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",深圳GDP首超香港？统计局辟谣：未超但差距再度缩小,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP overtakes Hong Kong? Bureau of ...,unrelated
6,6,9,10,"""吃榴莲的禁忌,吃错会致命!","榴莲不能和什么一起吃 与咖啡同吃诱发心脏病""""","""if you eat durian, you will kill yourself if ...","Durian can't eat with anything, it's the same ...",unrelated
7,5,2,11,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",深圳GDP首超香港？辟谣：未超但差距再度缩小,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP outpaces Hong Kong? Defending R...,unrelated
8,7,12,13,"""旅行青蛙？居然是一款""""生育意愿测试器”！大家还是玩""""珠宝V课""""吧""",咸宁一家店的蛋糕含有“棉花”？崇阳多部门联合辟谣,"""Frog frog? It's a fertility test! Let's play""...","A store in xianning contains ""cotton""? A multi...",unrelated
9,8,6,14,"""用大蒜鉴别地沟油的方法,怎么鉴别地沟油",一颗大蒜就能鉴别地沟油？别闹了！做到下面几点，让您远离地沟油,"""How to discriminate oil from gutter oil by me...",A single piece of garlic can spot gutter oil? ...,agreed


In [10]:
df_samplesub.head(10)

,Id,Category
0,347448,unrelated
1,347449,unrelated
2,359100,unrelated
3,359101,unrelated
4,359102,unrelated
5,359103,unrelated
6,359104,unrelated
7,359105,unrelated
8,359106,unrelated
9,359107,unrelated


In [9]:
df_train['label'].value_counts()

unrelated    219313
agreed        92973
disagreed      8266
Name: label, dtype: int64

In [12]:
#encoding=utf-8
import jieba

In [13]:
seg_list = jieba.cut("2017养老保险又新增两项，农村老人人人可申领，你领到了吗", cut_all=False)
print(list(seg_list))

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.993 seconds.
Prefix dict has been built succesfully.


['2017', '养老保险', '又', '新增', '两项', '，', '农村', '老人', '人人', '可', '申领', '，', '你', '领到', '了', '吗']


In [ ]:
df = df_train

In [16]:
df_train.shape, df_test.shape

((320552, 8), (80126, 7))

In [18]:
dictt = {}

for df in [df_train, df_test]:
    for i in df[['tid1','title1_zh']].drop_duplicates().iterrows():
        dictt[i[1]['tid1']] = i[1]['title1_zh']
    for i in df[['tid2','title2_zh']].drop_duplicates().iterrows():
        dictt[i[1]['tid2']] = i[1]['title2_zh']

In [19]:
len(dictt)

189305

In [20]:
dictt2 = {}
for i in dictt.keys():
    try:
        seg_list = jieba.cut(dictt[i], cut_all=False)
        dictt2[i] = list(seg_list)
    except AttributeError:
        dictt2[i] = [] 

In [ ]:
import numpy as np
from gensim.models import KeyedVectors


tmp_file = '../data/merge_sgns_bigram_char300.txt'
model = KeyedVectors.load_word2vec_format(tmp_file)
model.get_vector('的')

In [34]:
aafa = []
for l in dictt2.values():
    aafa.append(len(l))
    
np.median(aafa), np.percentile(aafa, q=99.5), np.mean(aafa), np.max(aafa)

(15.0, 26.0, 14.929119674599193, 95)

In [36]:
import tensorflow as tf
import keras
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

In [37]:
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

In [43]:
word_embeddings = {i:j for i,j in zip(model.wv.index2word, list(model.wv.vectors))}
word_index = {j:i for i,j in enumerate(word_embeddings.keys())}

/home/dmlab/ksedm1/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [79]:
dictt3 = {}
word_thres = int(np.percentile(aafa, q=99.5))
for i,j in dictt2.items():
    if len(j) < word_thres:
        k = j + ['']*(word_thres-len(j))
    else:
        k = j[:word_thres]
    dictt3[i] = k

In [120]:
df_train['label'] = pd.factorize(df_train['label'])[0]

In [122]:
df_train.head(5)

,id,tid1,tid2,title1_zh,title2_zh,title1_en,title2_en,label
0,0,0,1,2017养老保险又新增两项，农村老人人人可申领，你领到了吗,警方辟谣“鸟巢大会每人领5万” 仍有老人坚持进京,There are two new old-age insurance benefits f...,"Police disprove ""bird's nest congress each per...",0
1,3,2,3,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",深圳GDP首超香港？深圳统计局辟谣：只是差距在缩小,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP outstrips Hong Kong? Shenzhen S...,0
2,1,2,4,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",GDP首超香港？深圳澄清：还差一点点……,"""If you do not come to Shenzhen, sooner or lat...",The GDP overtopped Hong Kong? Shenzhen clarifi...,0
3,2,2,5,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",去年深圳GDP首超香港？深圳统计局辟谣：还差611亿,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP topped Hong Kong last year? She...,0
4,9,6,7,"""用大蒜鉴别地沟油的方法,怎么鉴别地沟油",吃了30年食用油才知道，一片大蒜轻松鉴别地沟油,"""How to discriminate oil from gutter oil by me...",It took 30 years of cooking oil to know that o...,1


In [402]:
class Prediction:
    def __init__(self):
        self.batch_size = 128
        self._load_word_embedding()
        self._train_val_split()
    
    
    def _load_word_embedding(self):
        self.word_embedding = word_embeddings

        
    def _simple_attention(self, target, reference):
        attention = keras.layers.Dense(1, activation=keras.activations.tanh)(reference)
        attention = keras.layers.Reshape((-1,))(attention)
        attention = keras.layers.Activation(keras.activations.softmax)(attention)
        return keras.layers.Dot((1,1))([target, attention])
    
        
    def _train_val_split(self):
        idxs = list(df_train.index)
        np.random.shuffle(idxs)
        val_num = len(idxs) // 5 
        self.train_idxs = idxs[:-val_num]
        self.val_idxs = idxs[-val_num:]
        self.training_data_size = len(self.train_idxs)
        self.validation_data_size = len(self.val_idxs)
        
        
    def _build_model(self):
        max_num_word = np.percentile(aafa, q=99.5)
        word_embedding_layer = keras.layers.Embedding(
            input_dim = len(self.word_embedding),
            output_dim = len(list(self.word_embedding.values())[0]),
            weights = [np.array(list(self.word_embedding.values()))],
            input_length = max_num_word,
            trainable = False
        )
        
        dropout = keras.layers.Dropout(0.2)
        
        word_input = keras.Input((max_num_word,))
        word_emb = word_embedding_layer(word_input)
        word_cnn = keras.layers.Conv1D(filters=200, kernel_size=5, padding='same', activation='relu', strides=1)(word_emb)
        word_att = self._simple_attention(word_cnn, word_cnn)
        

        word_input2 = keras.Input((max_num_word,))
        word_emb2 = word_embedding_layer(word_input2)
        word_cnn2 = keras.layers.Conv1D(filters=200, kernel_size=5, padding='same', activation='relu', strides=1)(word_emb2)
        word_att2 = self._simple_attention(word_cnn2, word_cnn2)
        
        word_output = keras.layers.Concatenate()([word_att, word_att2])
        
        
        logits = keras.layers.Dense(3, activation=keras.activations.softmax)(word_output)
        model = keras.Model([word_input, word_input2], logits)
        model.compile(
            optimizer = keras.optimizers.Adam(0.001),
            loss = keras.losses.categorical_crossentropy,
            metrics = [keras.metrics.categorical_accuracy]
        )
        return model

    
    def _training_data_generator(self):
        def __gen__():
            while True:
                for idx in self.train_idxs:
                    pair = df_train.iloc[idx]
                    text1_ind = [word_index.get(x, 893405) for x in dictt3[pair['tid1']]]
                    text2_ind = [word_index.get(x, 893405) for x in dictt3[pair['tid2']]]
                    yield text1_ind, text2_ind, pair['label']  
                    
        gen = __gen__()
        
        while True:
            batch = [np.stack(x) for x in zip(*(next(gen) for _ in range(self.batch_size)))]
            yield [batch[0], batch[1]], keras.utils.to_categorical(batch[-1], 3)
            
    
    
    def _validation_data_generator(self):
        def __gen__():
            while True:
                for idx in self.val_idxs:
                    pair = df_train.iloc[idx]
                    text1_ind = [word_index.get(x, 893405) for x in dictt3[pair['tid1']]]
                    text2_ind = [word_index.get(x, 893405) for x in dictt3[pair['tid2']]]
                    yield text1_ind, text2_ind, pair['label']  
                    
        gen = __gen__()
        
        while True:
            batch = [np.stack(x) for x in zip(*(next(gen) for _ in range(self.batch_size)))]
            yield [batch[0], batch[1]], keras.utils.to_categorical(batch[-1], 3)
                
    
    
    def _test_data_generator(self):
        def __gen__():
            while True:
                idxs = list(df_test.index)
                for idx in idxs:
                    pair = df_test.iloc[idx]
                    text1_ind = [word_index.get(x, 893405) for x in dictt3[pair['tid1']]]
                    text2_ind = [word_index.get(x, 893405) for x in dictt3[pair['tid2']]]
                    yield text1_ind, text2_ind, pair['label']  
                    
        gen = __gen__()
        
        while True:
            batch = [np.stack(x) for x in zip(*(next(gen) for _ in range(len(df_test))))]
            yield [batch[0], batch[1]]  
            
    
    def _test_batch_data(self):
        t1 = df_test['tid1'].apply(lambda x: [word_index.get(y, 893405) for y in dictt3[x]]).apply(np.array)
        t2 = df_test['tid2'].apply(lambda x: [word_index.get(y, 893405) for y in dictt3[x]]).apply(np.array)
        return [np.array(list(t1)), np.array(list(t2))]
   
    
    
    def train(self):
        batch_size = 20
        self.training_data = self._training_data_generator()
        self.validation_data = self._validation_data_generator()
        #         self.test_data = self._test_data_generator()
        self.test_data = self._test_batch_data()
        
        self.model = self._build_model()
        
        self.history = self.model.fit_generator(
            generator = self.training_data,
            validation_data = self.validation_data,
            steps_per_epoch = self.training_data_size // self.batch_size,
            validation_steps = self.validation_data_size // self.batch_size,
            class_weight = {0:1/16, 1:1/15, 2:1/5},
            epochs = 3
        )
      
        
        self.result = self.model.predict_on_batch(self.test_data)
           
#         self.result = self.model.predict_generator(
#             generator = self.test_data,
#             steps = 1
#         )
        

In [403]:
r = Prediction()
r.train()

Epoch 1/3
2003/2003 [==============================] - 109s 55ms/step - loss: 0.0361 - categorical_accuracy: 0.7531 - val_loss: 0.4294 - val_categorical_accuracy: 0.7951
Epoch 2/3
2003/2003 [==============================] - 107s 54ms/step - loss: 0.0282 - categorical_accuracy: 0.8196 - val_loss: 0.4062 - val_categorical_accuracy: 0.8121
Epoch 3/3
2003/2003 [==============================] - 108s 54ms/step - loss: 0.0239 - categorical_accuracy: 0.8519 - val_loss: 0.3941 - val_categorical_accuracy: 0.8224


In [ ]:
## validation accuacy가 가장 높은 epoch의 모델로 test data를 평가하는 걸로 코드 전환 - 모델 save or stopping condition

In [404]:
df_test['pred_label'] = r.result.argmax(axis=1)
idx_label = {0:'unrelated', 1:'agreed', 2:'disagreed'}
df_test['pred_label'] = df_test['pred_label'].apply(lambda x: idx_label[x])
print(df_test['pred_label'].value_counts())


unrelated    56037
agreed       22021
disagreed     2068
Name: pred_label, dtype: int64


In [405]:
submission_path = '../data/sample_submission_sd_weighted.csv'
df_test[['id', 'pred_label']].rename(columns={'id':'Id','pred_label':'Category'}).to_csv(submission_path, index=False)

In [406]:
df_train['label'].value_counts()

0    219313
1     92973
2      8266
Name: label, dtype: int64

In [418]:
min(df_train.tid1), max(df_train.tid1), min(df_test.tid1), max(df_test.tid1)

(0, 167563, 17, 189305)

In [419]:
min(df_train.tid2), max(df_train.tid2), min(df_test.tid2), max(df_test.tid2)

(1, 167558, 3, 189303)